In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Potato Disease Classification"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Dataset credits: https://www.kaggle.com/arjuntejaswi/plant-village"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Import all the Dependencies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import tensorflow as tf\n",
    "from tensorflow.keras import models, layers\n",
    "import matplotlib.pyplot as plt\n",
    "from IPython.display import HTML"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Set all the Constants"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "BATCH_SIZE = 32\n",
    "IMAGE_SIZE = 256\n",
    "CHANNELS=3\n",
    "EPOCHS=50"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Import data into tensorflow dataset object"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We will use image_dataset_from_directory api to load all images in tensorflow dataset: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Found 2152 files belonging to 3 classes.\n"
     ]
    }
   ],
   "source": [
    "dataset = tf.keras.preprocessing.image_dataset_from_directory(\n",
    "    \"PlantVillage\",\n",
    "    seed=123,\n",
    "    shuffle=True,\n",
    "    image_size=(IMAGE_SIZE,IMAGE_SIZE),\n",
    "    batch_size=BATCH_SIZE\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Watch below video on tensorflow input pipeline first if you don't know about tensorflow datasets**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "<iframe width=\"560\" height=\"315\" src=\"https://www.youtube.com/embed/VFEOskzhhbc\" title=\"YouTube video player\" frameborder=\"0\" allow=\"accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>\n"
      ],
      "text/plain": [
       "<IPython.core.display.HTML object>"
      ]
     },
     "execution_count": 4,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "HTML(\"\"\"\n",
    "<iframe width=\"560\" height=\"315\" src=\"https://www.youtube.com/embed/VFEOskzhhbc\" title=\"YouTube video player\" frameborder=\"0\" allow=\"accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>\n",
    "\"\"\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']"
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "class_names = dataset.class_names\n",
    "class_names"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "(32, 256, 256, 3)\n",
      "[1 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1 1 2 0 0]\n"
     ]
    }
   ],
   "source": [
    "for image_batch, labels_batch in dataset.take(1):\n",
    "    print(image_batch.shape)\n",
    "    print(labels_batch.numpy())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "As you can see above, each element in the dataset is a tuple. First element is a batch of 32 elements of images. Second element is a batch of 32 elements of class labels "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Visualize some of the images from our dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 720x720 with 12 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.figure(figsize=(10, 10))\n",
    "for image_batch, labels_batch in dataset.take(1):\n",
    "    for i in range(12):\n",
    "        ax = plt.subplot(3, 4, i + 1)\n",
    "        plt.imshow(image_batch[i].numpy().astype(\"uint8\"))\n",
    "        plt.title(class_names[labels_batch[i]])\n",
    "        plt.axis(\"off\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Function to Split Dataset\n",
    "\n",
    "Dataset should be bifurcated into 3 subsets, namely:\n",
    "1. Training: Dataset to be used while training\n",
    "2. Validation: Dataset to be tested against while training\n",
    "3. Test: Dataset to be tested against after we trained a model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "68"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(dataset)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "54.400000000000006"
      ]
     },
     "execution_count": 9,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "train_size = 0.8\n",
    "len(dataset)*train_size"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "54"
      ]
     },
     "execution_count": 10,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "train_ds = dataset.take(54)\n",
    "len(train_ds)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "14"
      ]
     },
     "execution_count": 11,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "test_ds = dataset.skip(54)\n",
    "len(test_ds)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "6.800000000000001"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "val_size=0.1\n",
    "len(dataset)*val_size"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "6"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "val_ds = test_ds.take(6)\n",
    "len(val_ds)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "8"
      ]
     },
     "execution_count": 14,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "test_ds = test_ds.skip(6)\n",
    "len(test_ds)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):\n",
    "    assert (train_split + test_split + val_split) == 1\n",
    "    \n",
    "    ds_size = len(ds)\n",
    "    \n",
    "    if shuffle:\n",
    "        ds = ds.shuffle(shuffle_size, seed=12)\n",
    "    \n",
    "    train_size = int(train_split * ds_size)\n",
    "    val_size = int(val_split * ds_size)\n",
    "    \n",
    "    train_ds = ds.take(train_size)    \n",
    "    val_ds = ds.skip(train_size).take(val_size)\n",
    "    test_ds = ds.skip(train_size).skip(val_size)\n",
    "    \n",
    "    return train_ds, val_ds, test_ds"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [],
   "source": [
    "train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "54"
      ]
     },
     "execution_count": 17,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(train_ds)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "6"
      ]
     },
     "execution_count": 18,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(val_ds)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "8"
      ]
     },
     "execution_count": 19,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(test_ds)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Cache, Shuffle, and Prefetch the Dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [],
   "source": [
    "train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)\n",
    "val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)\n",
    "test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Building the Model"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Creating a Layer for Resizing and Normalization\n",
    "Before we feed our images to network, we should be resizing it to the desired size. \n",
    "Moreover, to improve model performance, we should normalize the image pixel value (keeping them in range 0 and 1 by dividing by 256).\n",
    "This should happen while training as well as inference. Hence we can add that as a layer in our Sequential Model.\n",
    "\n",
    "You might be thinking why do we need to resize (256,256) image to again (256,256). You are right we don't need to but this will be useful when we are done with the training and start using the model for predictions. At that time somone can supply an image that is not (256,256) and this layer will resize it"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "resize_and_rescale = tf.keras.Sequential([\n",
    "  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),\n",
    "  layers.experimental.preprocessing.Rescaling(1./255),\n",
    "])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Data Augmentation\n",
    "Data Augmentation is needed when we have less data, this boosts the accuracy of our model by augmenting the data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [],
   "source": [
    "data_augmentation = tf.keras.Sequential([\n",
    "  layers.experimental.preprocessing.RandomFlip(\"horizontal_and_vertical\"),\n",
    "  layers.experimental.preprocessing.RandomRotation(0.2),\n",
    "])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Watch below video if you are not familiar with data augmentation**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "<iframe width=\"560\" height=\"315\" src=\"https://www.youtube.com/embed/mTVf7BN7S8w\" title=\"YouTube video player\" frameborder=\"0\" allow=\"accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>\n"
      ],
      "text/plain": [
       "<IPython.core.display.HTML object>"
      ]
     },
     "execution_count": 23,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "HTML(\"\"\"\n",
    "<iframe width=\"560\" height=\"315\" src=\"https://www.youtube.com/embed/mTVf7BN7S8w\" title=\"YouTube video player\" frameborder=\"0\" allow=\"accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>\n",
    "\"\"\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Model Architecture\n",
    "We use a CNN coupled with a Softmax activation in the output layer. We also add the initial layers for resizing, normalization and Data Augmentation."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**We are going to use convolutional neural network (CNN) here. CNN is popular for image classification tasks. Watch below video to understand fundamentals of CNN**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "<iframe width=\"560\" height=\"315\" src=\"https://www.youtube.com/embed/zfiSAzpy9NM\" title=\"YouTube video player\" frameborder=\"0\" allow=\"accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>\n"
      ],
      "text/plain": [
       "<IPython.core.display.HTML object>"
      ]
     },
     "execution_count": 24,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "HTML(\"\"\"\n",
    "<iframe width=\"560\" height=\"315\" src=\"https://www.youtube.com/embed/zfiSAzpy9NM\" title=\"YouTube video player\" frameborder=\"0\" allow=\"accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>\n",
    "\"\"\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "metadata": {},
   "outputs": [],
   "source": [
    "input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)\n",
    "n_classes = 3\n",
    "\n",
    "model = models.Sequential([\n",
    "    resize_and_rescale,\n",
    "    data_augmentation,\n",
    "    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),\n",
    "    layers.MaxPooling2D((2, 2)),\n",
    "    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),\n",
    "    layers.MaxPooling2D((2, 2)),\n",
    "    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),\n",
    "    layers.MaxPooling2D((2, 2)),\n",
    "    layers.Conv2D(64, (3, 3), activation='relu'),\n",
    "    layers.MaxPooling2D((2, 2)),\n",
    "    layers.Conv2D(64, (3, 3), activation='relu'),\n",
    "    layers.MaxPooling2D((2, 2)),\n",
    "    layers.Conv2D(64, (3, 3), activation='relu'),\n",
    "    layers.MaxPooling2D((2, 2)),\n",
    "    layers.Flatten(),\n",
    "    layers.Dense(64, activation='relu'),\n",
    "    layers.Dense(n_classes, activation='softmax'),\n",
    "])\n",
    "\n",
    "model.build(input_shape=input_shape)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Model: \"sequential_2\"\n",
      "_________________________________________________________________\n",
      "Layer (type)                 Output Shape              Param #   \n",
      "=================================================================\n",
      "sequential (Sequential)      (32, 256, 256, 3)         0         \n",
      "_________________________________________________________________\n",
      "sequential_1 (Sequential)    (32, 256, 256, 3)         0         \n",
      "_________________________________________________________________\n",
      "conv2d (Conv2D)              (32, 254, 254, 32)        896       \n",
      "_________________________________________________________________\n",
      "max_pooling2d (MaxPooling2D) (32, 127, 127, 32)        0         \n",
      "_________________________________________________________________\n",
      "conv2d_1 (Conv2D)            (32, 125, 125, 64)        18496     \n",
      "_________________________________________________________________\n",
      "max_pooling2d_1 (MaxPooling2 (32, 62, 62, 64)          0         \n",
      "_________________________________________________________________\n",
      "conv2d_2 (Conv2D)            (32, 60, 60, 64)          36928     \n",
      "_________________________________________________________________\n",
      "max_pooling2d_2 (MaxPooling2 (32, 30, 30, 64)          0         \n",
      "_________________________________________________________________\n",
      "conv2d_3 (Conv2D)            (32, 28, 28, 64)          36928     \n",
      "_________________________________________________________________\n",
      "max_pooling2d_3 (MaxPooling2 (32, 14, 14, 64)          0         \n",
      "_________________________________________________________________\n",
      "conv2d_4 (Conv2D)            (32, 12, 12, 64)          36928     \n",
      "_________________________________________________________________\n",
      "max_pooling2d_4 (MaxPooling2 (32, 6, 6, 64)            0         \n",
      "_________________________________________________________________\n",
      "conv2d_5 (Conv2D)            (32, 4, 4, 64)            36928     \n",
      "_________________________________________________________________\n",
      "max_pooling2d_5 (MaxPooling2 (32, 2, 2, 64)            0         \n",
      "_________________________________________________________________\n",
      "flatten (Flatten)            (32, 256)                 0         \n",
      "_________________________________________________________________\n",
      "dense (Dense)                (32, 64)                  16448     \n",
      "_________________________________________________________________\n",
      "dense_1 (Dense)              (32, 3)                   195       \n",
      "=================================================================\n",
      "Total params: 183,747\n",
      "Trainable params: 183,747\n",
      "Non-trainable params: 0\n",
      "_________________________________________________________________\n"
     ]
    }
   ],
   "source": [
    "model.summary()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Compiling the Model\n",
    "We use `adam` Optimizer, `SparseCategoricalCrossentropy` for losses, `accuracy` as a metric"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 27,
   "metadata": {},
   "outputs": [],
   "source": [
    "model.compile(\n",
    "    optimizer='adam',\n",
    "    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),\n",
    "    metrics=['accuracy']\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 28,
   "metadata": {
    "scrolled": false
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Epoch 1/50\n",
      "54/54 [==============================] - 17s 53ms/step - loss: 0.9088 - accuracy: 0.5041 - val_loss: 0.8736 - val_accuracy: 0.4896\n",
      "Epoch 2/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.7452 - accuracy: 0.6644 - val_loss: 0.8316 - val_accuracy: 0.6510\n",
      "Epoch 3/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.5177 - accuracy: 0.7766 - val_loss: 0.4071 - val_accuracy: 0.8490\n",
      "Epoch 4/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.3338 - accuracy: 0.8617 - val_loss: 0.3461 - val_accuracy: 0.8385\n",
      "Epoch 5/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.3438 - accuracy: 0.8513 - val_loss: 0.4481 - val_accuracy: 0.8125\n",
      "Epoch 6/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.2661 - accuracy: 0.8958 - val_loss: 0.1775 - val_accuracy: 0.9375\n",
      "Epoch 7/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.2115 - accuracy: 0.9207 - val_loss: 0.1383 - val_accuracy: 0.9531\n",
      "Epoch 8/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.1419 - accuracy: 0.9439 - val_loss: 0.3838 - val_accuracy: 0.8906\n",
      "Epoch 9/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.1470 - accuracy: 0.9421 - val_loss: 0.1647 - val_accuracy: 0.9427\n",
      "Epoch 10/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.1137 - accuracy: 0.9601 - val_loss: 0.1253 - val_accuracy: 0.9427\n",
      "Epoch 11/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0701 - accuracy: 0.9722 - val_loss: 0.0548 - val_accuracy: 0.9792\n",
      "Epoch 12/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.1593 - accuracy: 0.9381 - val_loss: 0.0971 - val_accuracy: 0.9740\n",
      "Epoch 13/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0896 - accuracy: 0.9705 - val_loss: 0.0366 - val_accuracy: 0.9844\n",
      "Epoch 14/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0933 - accuracy: 0.9641 - val_loss: 0.0604 - val_accuracy: 0.9792\n",
      "Epoch 15/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0464 - accuracy: 0.9838 - val_loss: 0.1363 - val_accuracy: 0.9635\n",
      "Epoch 16/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0444 - accuracy: 0.9850 - val_loss: 0.1005 - val_accuracy: 0.9688\n",
      "Epoch 17/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0681 - accuracy: 0.9745 - val_loss: 0.0833 - val_accuracy: 0.9688\n",
      "Epoch 18/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.1049 - accuracy: 0.9618 - val_loss: 0.0943 - val_accuracy: 0.9635\n",
      "Epoch 19/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0680 - accuracy: 0.9740 - val_loss: 0.0258 - val_accuracy: 0.9896\n",
      "Epoch 20/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0271 - accuracy: 0.9919 - val_loss: 0.0388 - val_accuracy: 0.9896\n",
      "Epoch 21/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0556 - accuracy: 0.9815 - val_loss: 0.1344 - val_accuracy: 0.9479\n",
      "Epoch 22/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0460 - accuracy: 0.9850 - val_loss: 0.0130 - val_accuracy: 0.9948\n",
      "Epoch 23/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0493 - accuracy: 0.9821 - val_loss: 0.0163 - val_accuracy: 0.9896\n",
      "Epoch 24/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0564 - accuracy: 0.9826 - val_loss: 0.0434 - val_accuracy: 0.9896\n",
      "Epoch 25/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0516 - accuracy: 0.9826 - val_loss: 0.0291 - val_accuracy: 0.9948\n",
      "Epoch 26/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0409 - accuracy: 0.9850 - val_loss: 0.0160 - val_accuracy: 1.0000\n",
      "Epoch 27/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0361 - accuracy: 0.9878 - val_loss: 0.0535 - val_accuracy: 0.9896\n",
      "Epoch 28/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0663 - accuracy: 0.9780 - val_loss: 0.1345 - val_accuracy: 0.9583\n",
      "Epoch 29/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0296 - accuracy: 0.9913 - val_loss: 0.0069 - val_accuracy: 1.0000\n",
      "Epoch 30/50\n",
      "54/54 [==============================] - 2s 30ms/step - loss: 0.0251 - accuracy: 0.9890 - val_loss: 0.0292 - val_accuracy: 0.9896\n",
      "Epoch 31/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0309 - accuracy: 0.9919 - val_loss: 0.2023 - val_accuracy: 0.9531\n",
      "Epoch 32/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0358 - accuracy: 0.9896 - val_loss: 0.0631 - val_accuracy: 0.9844\n",
      "Epoch 33/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0369 - accuracy: 0.9850 - val_loss: 0.0768 - val_accuracy: 0.9896\n",
      "Epoch 34/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0382 - accuracy: 0.9867 - val_loss: 0.1520 - val_accuracy: 0.9531\n",
      "Epoch 35/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0284 - accuracy: 0.9896 - val_loss: 0.5256 - val_accuracy: 0.8958\n",
      "Epoch 36/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0810 - accuracy: 0.9728 - val_loss: 0.1665 - val_accuracy: 0.9271\n",
      "Epoch 37/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0477 - accuracy: 0.9803 - val_loss: 0.0292 - val_accuracy: 0.9844\n",
      "Epoch 38/50\n",
      "54/54 [==============================] - 2s 29ms/step - loss: 0.0297 - accuracy: 0.9907 - val_loss: 0.0228 - val_accuracy: 0.9896\n",
      "Epoch 39/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0368 - accuracy: 0.9838 - val_loss: 0.0163 - val_accuracy: 0.9948\n",
      "Epoch 40/50\n",
      "54/54 [==============================] - 1s 28ms/step - loss: 0.0272 - accuracy: 0.9907 - val_loss: 0.0148 - val_accuracy: 0.9948\n",
      "Epoch 41/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0178 - accuracy: 0.9925 - val_loss: 0.0076 - val_accuracy: 1.0000\n",
      "Epoch 42/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0563 - accuracy: 0.9803 - val_loss: 0.0299 - val_accuracy: 0.9844\n",
      "Epoch 43/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0222 - accuracy: 0.9907 - val_loss: 0.0102 - val_accuracy: 0.9948\n",
      "Epoch 44/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0261 - accuracy: 0.9896 - val_loss: 0.0909 - val_accuracy: 0.9688\n",
      "Epoch 45/50\n",
      "54/54 [==============================] - 1s 28ms/step - loss: 0.0181 - accuracy: 0.9936 - val_loss: 0.0747 - val_accuracy: 0.9688\n",
      "Epoch 46/50\n",
      "54/54 [==============================] - 1s 28ms/step - loss: 0.0119 - accuracy: 0.9942 - val_loss: 0.0258 - val_accuracy: 0.9948\n",
      "Epoch 47/50\n",
      "54/54 [==============================] - 1s 28ms/step - loss: 0.0256 - accuracy: 0.9907 - val_loss: 0.0777 - val_accuracy: 0.9740\n",
      "Epoch 48/50\n",
      "54/54 [==============================] - 1s 28ms/step - loss: 0.0262 - accuracy: 0.9913 - val_loss: 0.0515 - val_accuracy: 0.9896\n",
      "Epoch 49/50\n",
      "54/54 [==============================] - 2s 28ms/step - loss: 0.0320 - accuracy: 0.9896 - val_loss: 0.0082 - val_accuracy: 1.0000\n",
      "Epoch 50/50\n",
      "54/54 [==============================] - 1s 28ms/step - loss: 0.0069 - accuracy: 0.9977 - val_loss: 0.0056 - val_accuracy: 1.0000\n"
     ]
    }
   ],
   "source": [
    "history = model.fit(\n",
    "    train_ds,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    validation_data=val_ds,\n",
    "    verbose=1,\n",
    "    epochs=50,\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "8/8 [==============================] - 1s 9ms/step - loss: 0.0138 - accuracy: 0.9961\n"
     ]
    }
   ],
   "source": [
    "scores = model.evaluate(test_ds)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**You can see above that we get 98.83% accuracy for our test dataset. This is considered to be a pretty good accuracy**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[0.013836881145834923, 0.99609375]"
      ]
     },
     "execution_count": 30,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "scores"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Scores is just a list containing loss and accuracy value"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Saving the Model\n",
    "We append the model to the list of models as a new version"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 31,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "INFO:tensorflow:Assets written to: ../models/3\\assets\n"
     ]
    }
   ],
   "source": [
    "import os\n",
    "model_version=max([int(i) for i in os.listdir(\"../models\") + [0]])+1\n",
    "model.save(f\"../models/{model_version}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Convert saved model to tf lite"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 98,
   "metadata": {},
   "outputs": [],
   "source": [
    "converter = tf.lite.TFLiteConverter.from_saved_model(\"../models/3\")\n",
    "converter.optimizations = [tf.lite.Optimize.DEFAULT]\n",
    "tflite_model = converter.convert()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Refer to this documentation for how to run inference on tflite model\n",
    "\n",
    "https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 99,
   "metadata": {},
   "outputs": [],
   "source": [
    "interpreter = tf.lite.Interpreter(model_content=tflite_model)\n",
    "interpreter.allocate_tensors()\n",
    "\n",
    "input_details = interpreter.get_input_details()\n",
    "output_details = interpreter.get_output_details()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 100,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[{'name': 'serving_default_sequential_input:0',\n",
       "  'index': 0,\n",
       "  'shape': array([  1, 256, 256,   3]),\n",
       "  'shape_signature': array([ -1, 256, 256,   3]),\n",
       "  'dtype': numpy.float32,\n",
       "  'quantization': (0.0, 0),\n",
       "  'quantization_parameters': {'scales': array([], dtype=float32),\n",
       "   'zero_points': array([], dtype=int32),\n",
       "   'quantized_dimension': 0},\n",
       "  'sparsity_parameters': {}}]"
      ]
     },
     "execution_count": 100,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "input_details"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 101,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[{'name': 'StatefulPartitionedCall:0',\n",
       "  'index': 35,\n",
       "  'shape': array([1, 3]),\n",
       "  'shape_signature': array([-1,  3]),\n",
       "  'dtype': numpy.float32,\n",
       "  'quantization': (0.0, 0),\n",
       "  'quantization_parameters': {'scales': array([], dtype=float32),\n",
       "   'zero_points': array([], dtype=int32),\n",
       "   'quantized_dimension': 0},\n",
       "  'sparsity_parameters': {}}]"
      ]
     },
     "execution_count": 101,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "output_details"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 102,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<PrefetchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>"
      ]
     },
     "execution_count": 102,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "test_ds"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 103,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "<TakeDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>"
      ]
     },
     "execution_count": 103,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "test_ds.take(1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 104,
   "metadata": {},
   "outputs": [],
   "source": [
    "input_index = interpreter.get_input_details()[0][\"index\"]\n",
    "output_index = interpreter.get_output_details()[0][\"index\"]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 107,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "tf.Tensor([1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1], shape=(32,), dtype=int32)\n",
      "tf.Tensor([0 1 0 0 1 0 2 0 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 2 1 1 0 1], shape=(32,), dtype=int32)\n",
      "tf.Tensor([0 0 1 0 0 1 0 1 0 1 0 0 0 1 1 2 0 0 1 0 1 1 1 0 0 0 0 1 1 2 0 1], shape=(32,), dtype=int32)\n",
      "tf.Tensor([1 0 1 1 1 0 1 2 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0], shape=(32,), dtype=int32)\n",
      "tf.Tensor([0 0 1 0 1 0 1 1 2 0 0 0 0 0 2 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 1 0], shape=(32,), dtype=int32)\n",
      "0\n"
     ]
    }
   ],
   "source": [
    "count = 0\n",
    "for image, label in test_ds.take(5):\n",
    "    print(label)\n",
    "    \n",
    "print(count)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 110,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "image shape (256, 256, 3)\n",
      "label tf.Tensor(0, shape=(), dtype=int32)\n",
      "image shape (256, 256, 3)\n",
      "label tf.Tensor(0, shape=(), dtype=int32)\n",
      "image shape (256, 256, 3)\n",
      "label tf.Tensor(1, shape=(), dtype=int32)\n",
      "image shape (256, 256, 3)\n",
      "label tf.Tensor(0, shape=(), dtype=int32)\n",
      "image shape (256, 256, 3)\n",
      "label tf.Tensor(1, shape=(), dtype=int32)\n"
     ]
    }
   ],
   "source": [
    "for image, label in test_ds.unbatch().take(5):\n",
    "    print(\"image shape\",image.shape)\n",
    "    print(\"label\",label)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 125,
   "metadata": {},
   "outputs": [],
   "source": [
    "for image, label in test_ds.unbatch().take(5):\n",
    "    interpreter.set_tensor(input_index, [image.numpy().astype(np.float32)])\n",
    "    interpreter.invoke()\n",
    "\n",
    "    output = interpreter.get_tensor(output_index)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 111,
   "metadata": {},
   "outputs": [],
   "source": [
    "predicted_labels = []\n",
    "actual_labels = []\n",
    "\n",
    "for image, label in test_ds.unbatch():\n",
    "    interpreter.set_tensor(input_index, [image])\n",
    "    interpreter.invoke()\n",
    "\n",
    "    output = interpreter.get_tensor(output_index)\n",
    "    predicted_labels.append(np.argmax(output[0]))\n",
    "    actual_labels.append(label.numpy())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 116,
   "metadata": {
    "scrolled": false
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[0, 1, 1, 0, 0, 0, 0, 0, 0, 1]"
      ]
     },
     "execution_count": 116,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "predicted_labels[:10]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 117,
   "metadata": {
    "scrolled": false
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[0, 1, 1, 0, 0, 0, 0, 0, 0, 1]"
      ]
     },
     "execution_count": 117,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "actual_labels[:10]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 118,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([ True,  True,  True,  True,  True,  True,  True,  True,  True,\n",
       "        True])"
      ]
     },
     "execution_count": 118,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "comparison_vector = np.array(predicted_labels) == np.array(actual_labels)\n",
    "comparison_vector[:10]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 97,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.6666666666666666"
      ]
     },
     "execution_count": 97,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "np.array([True,False,True]).mean()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 115,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.99609375"
      ]
     },
     "execution_count": 115,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "accuracy = comparison_vector.mean()\n",
    "accuracy"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This shows that accuracy of our tflite model is around 99% which is considered to be pretty good. Now we can deploy this model to GCP (Google Cloud Platform)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

In [ ]:
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8" />
    <link rel="icon" href="%PUBLIC_URL%/cblogo.png" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <meta name="theme-color" content="#000000" />
    <meta
      name="description"
      content="Web site created using create-react-app"
    />
    <link rel="apple-touch-icon" href="%PUBLIC_URL%/logo192.png" />
    <!--
      manifest.json provides metadata used when your web app is installed on a
      user's mobile device or desktop. See https://developers.google.com/web/fundamentals/web-app-manifest/
    -->
    <link rel="manifest" href="%PUBLIC_URL%/manifest.json" />
    <!--
      Notice the use of %PUBLIC_URL% in the tags above.
      It will be replaced with the URL of the `public` folder during the build.
      Only files inside the `public` folder can be referenced from the HTML.

      Unlike "/favicon.ico" or "favicon.ico", "%PUBLIC_URL%/favicon.ico" will
      work correctly both with client-side routing and a non-root public URL.
      Learn how to configure a non-root public URL by running `npm run build`.
    -->
    <title>codebasics</title>
  </head>
  <body>
    <noscript>You need to enable JavaScript to run this app.</noscript>
    <div id="root"></div>
    <!--
      This HTML file is a template.
      If you open it directly in the browser, you will see an empty page.

      You can add webfonts, meta tags, or analytics to this file.
      The build step will place the bundled scripts into the <body> tag.

      To begin the development, run `npm start` or `yarn start`.
      To create a production bundle, use `npm run build` or `yarn build`.
    -->
  </body>
</html>